### Setup

In [1]:
import os, sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='cobra')

from cobra import io, Model, Solution
from scripts.helpers.model import rxn_in_model, met_in_model, add_single_gene_reaction_pair
from scripts.opt._fba import flux_balance_analysis
from scripts.opt._fva import run_flux_variability_analysis

In [52]:
params = {
    'mode': 'auto',
    'chloroplast': True
}

# Gap-filled model
model = io.read_sbml_model(f'../data/altered/xmls/MNL_iCre1355_{params['mode']}_GAPFILL/{"h/" if params['chloroplast'] else ""}SQS+SQE+MVA.xml')

# Add exchange reaction
ERG = "ergosterol_c"
ERGEXCH = "ERGOSTEROLEXCH"
if not rxn_in_model(model, ERGEXCH):
    add_single_gene_reaction_pair(
        model=model, 
        gene_id="EXCHERG_GENE",
        reaction_id=ERGEXCH,
        reaction_name="Ergosterol exchange (assumption)", 
        reaction_subsystem="Exchange", 
        metabolites=[(-1, ERG)],
        reversible=True
    )

No objective coefficients in model. Unclear what should be optimized


### FVA (Blocked)

In [53]:
fva = run_flux_variability_analysis(
    model,
    loopless=True,
    pfba_factor=None,
    objectives=['ALT_MVAS'],
    reactions=['ALT_MVAS', 'ALT_MVAD', 'ALT_MVAE', 'ALT_PMK', 'ALT_MVK', 'ALT_IDLI']
)

fva

,minimum,maximum
ALT_MVAS,0.0,-5.349016e-31
ALT_MVAD,0.0,0.000000e+00
ALT_MVAE,0.0,-5.349016e-31
ALT_PMK,0.0,0.000000e+00
ALT_MVK,0.0,-2.790923e-29
ALT_IDLI,0.0,1.379310e-01


### FVA (Repaired)

In [ ]:
temp = model.copy()

# Acetyl_CoA_c = temp.metabolites.get_by_id("accoa_c")
# Acetyl_CoA_h = temp.metabolites.get_by_id("accoa_h")

FPPS = temp.reactions.get_by_id('FPPS')
mets = FPPS.metabolites

add_single_gene_reaction_pair(
    model=temp,
    gene_id="FPPS_ATTEMPT",
    reaction_id='FPPS_NEW',
    reaction_name="Farnesyl pyrophosphate synthase (manual)",
    reaction_subsystem="Biosynthesis of steroids",
    metabolites=[ (k, v.id) for v, k in mets.items() ],
    reversible=False
)

FPPS.lower_bound = 0
FPPS.upper_bound = 0

fva = run_flux_variability_analysis(
    model=temp,
    loopless=True,
    pfba_factor=None,
    objectives=['ALT_MVAS'],
    reactions=['ALT_MVAS', 'ALT_MVAD', 'ALT_MVAE', 'ALT_PMK', 'ALT_MVK', 'ALT_IDLI']
)

fva

,minimum,maximum
ALT_MVAS,0.0,1.333895e-34
ALT_MVAD,0.0,0.000000e+00
ALT_MVAE,0.0,1.333895e-34
ALT_PMK,0.0,3.381919e-32
ALT_MVK,0.0,1.333895e-34
ALT_IDLI,0.0,1.379310e-01


### FVA (Knock-out)

In [54]:
temp2 = model.copy()

knockout = ['IDS1', 'IDS2', 'HDS', 'MCS', 'CMK', 'CMS', 'DXRI', 'DXS']
for k in knockout:
    rxn = temp2.reactions.get_by_id(k)
    rxn.lower_bound = 0
    rxn.upper_bound = 0
# alt = ['FRDPth', 'GRDPth', 'IPDPth']
# for rid in alt:
#     rxn = temp2.reactions.get_by_id(rid)
#     rxn.lower_bound = 0
#     rxn.upper_bound = 0

# ACCOAth = temp2.reactions.get_by_id('ACCOAth')
# ACCOAth.lower_bound = -0.10
# ACCOAth.upper_bound = -0.10

# ACACT = temp2.reactions.get_by_id('ACACT')
# ACACT.lower_bound = 0.05
# ACACT.upper_bound = 0.05
# ALT_MVAS = temp2.reactions.get_by_id('ALT_MVAS')
# ALT_MVAS.lower_bound = 0.05
# ALT_MVAS.upper_bound = 0.05

fva = run_flux_variability_analysis(
    temp2,
    loopless=True,
    pfba_factor=None,
    objectives=[ERGEXCH],
    reactions=[
        'ALT_MVAS',
        'ALT_MVAD',
        'ALT_MVAE',
        'ALT_PMK',
        'ALT_MVK',
        'ALT_IDLI',
        'GPPS',
        'FPPS',
        'SS',
        'ALT_SQS2',
        'Biomass_Chlamy_auto',
        ERGEXCH,
    ]
)

fva

,minimum,maximum
ALT_MVAS,0.0,-1.178412e-30
ALT_MVAD,0.0,0.000000e+00
ALT_MVAE,0.0,-1.178412e-30
ALT_PMK,0.0,-1.160930e-30
ALT_MVK,0.0,-4.063254e-30
ALT_IDLI,0.0,1.697661e-31
GPPS,0.0,3.390325e-31
FPPS,0.0,3.390325e-31
SS,0.0,-1.253351e-31
ALT_SQS2,0.0,1.639191e-31


In [55]:
os.makedirs(os.path.join('.', 'temp', 'mva'), exist_ok=True)
io.write_sbml_model(temp, os.path.join('.', 'temp', 'mva', 'temp.xml'))
io.write_sbml_model(temp2, os.path.join('.', 'temp', 'mva', 'temp2.xml'))

In [41]:
atp = temp2.reactions.get_by_id('ATP_3h_th')
accoath = temp2.reactions.get_by_id('ACCOAth')
print(atp.lower_bound, atp.upper_bound)
print(accoath.lower_bound, accoath.upper_bound)

0.0 1000.0
-1000.0 1000.0


### FVA (Sterol)